## Section 1. Introduction ##

In this notebook, the dataset to be processed is the Labor Force Survey conducted April 2016 and retrieved through Philippine Statistics Authority database. 



In [ ]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


plt.rcParams['figure.figsize'] = (6.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

plt.style.use('ggplot')

# autoreload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

<h1>Importing LFS PUF April 2016.CSV</h1>

In [ ]:
try:
    lfs_data = pd.read_csv("src/data/LFS PUF April 2016.CSV")
except FileNotFoundError:
    print("Error: CSV file not found. Please make sure the file exists in the correct directory or provide the correct path.")
    exit()

<h1>Data Information, Pre-Processing, and Cleaning</h1>

Let's get an overview of our dataset.

In [ ]:
lfs_data.info()

---
Of interest to us, there are:
<ul><li>1 contains float values, </li>
<li>14 contain integer values, and </li>
<li><b>35 are object values</b>.</li></ul>


---
Let's check for duplicates:

In [ ]:
lfs_data.duplicated().sum()

No duplicates here, and therefore no cleaning need follow in this regard.

The dataset seems to contain null values in the form of whitespaces. Let's count those:

In [ ]:
has_null = lfs_data.apply(lambda col: col.str.isspace().sum() if col.dtype == 'object' else 0)

print("Number Empty Cells:")
print(has_null[has_null > 0])

---
And standardize, replacing these whitespace values with -1:

In [ ]:
lfs_data.replace(r"^\s+$", -1, regex=True, inplace=True)
nan_counts_per_column = lfs_data.isna().sum()
print(nan_counts_per_column[nan_counts_per_column > 0])

Now that these are -1, let's return to the data types, and find if our object columns from earlier are convertible to integers (or float):

In [ ]:
int_convertible_columns = []

for col in lfs_data.columns:
    if lfs_data[col].dtypes == 'object':  
        try:
            float_vals = lfs_data[col].dropna().astype(float)
            if (float_vals % 1 == 0).all():
                int_convertible_columns.append(col)
        except ValueError:
            pass 

print("Safely convertable to int:")
print(int_convertible_columns)

---
And convert to int

In [ ]:
columns_to_convert = [
    'PUFC06_MSTAT', 'PUFC08_CURSCH', 'PUFC09_GRADTECH', 'PUFC10_CONWR', 'PUFC11_WORK', 
    'PUFC12_JOB', 'PUFC14_PROCC', 'PUFC16_PKB', 'PUFC17_NATEM', 'PUFC18_PNWHRS', 
    'PUFC19_PHOURS', 'PUFC20_PWMORE', 'PUFC21_PLADDW', 'PUFC22_PFWRK', 'PUFC23_PCLASS', 
    'PUFC24_PBASIS', 'PUFC25_PBASIC', 'PUFC26_OJOB', 'PUFC27_NJOBS', 'PUFC28_THOURS', 
    'PUFC29_WWM48H', 'PUFC30_LOOKW', 'PUFC31_FLWRK', 'PUFC32_JOBSM', 'PUFC33_WEEKS', 
    'PUFC34_WYNOT', 'PUFC35_LTLOOKW', 'PUFC36_AVAIL', 'PUFC37_WILLING', 'PUFC38_PREVJOB', 
    'PUFC40_POCC', 'PUFC41_WQTR', 'PUFC43_QKB', 'PUFNEWEMPSTAT'
]

for col in columns_to_convert:
    lfs_data[col] = lfs_data[col].astype(int) 

---
Let's also apply the unique() function to our dataset.

In [ ]:
lfs_data.apply(lambda x: x.nunique())

---
Considering our dataset has 18,000 entries, features with particularly low numbers stand out as questions that have clear, defined choices. Reviewing the [questionnaire](https://psada.psa.gov.ph/catalog/67/download/537), we find that certain questions ask the participant to specify beyond prespecified choices.

This column possibly contains "010," which is obviously not an integer. We ensure this column is a string, and check for values not specified in the questionnaire.

In [ ]:
lfs_data['PUFC07_GRADE'] = lfs_data['PUFC07_GRADE']
valid_codes = [
    0, 10,  # No Grade, Preschool
    210, 220, 230, 240, 250, 260, 280,  # Elementary
    310, 320, 330, 340, 350,  # High School
    410, 420,  # Post Secondary; If Graduate Specify
    810, 820, 830, 840,  # College; If Graduate Specify
    900,  # Post Baccalaureate
    np.nan
]
invalid_rows = lfs_data[~(lfs_data['PUFC07_GRADE'].isin(valid_codes))]

unique_invalid_values = invalid_rows['PUFC07_GRADE'].unique()
print(unique_invalid_values)

Values 5XX 6XX are not detailed in the questionnaire. As it instructs the participant to specify whether they graduated from post secondary or college, we'll create a new data point to encapsulate these.

In [ ]:
lfs_data.loc[~lfs_data['PUFC07_GRADE'].isin(valid_codes), 'PUFC07_GRADE'] = 700
print(lfs_data['PUFC07_GRADE'].unique())

In [ ]:
lfs_data['PUFC11_WORK'].value_counts()

## EDA

In [ ]:
lfs_data_with_nan = lfs_data.copy()
lfs_data_with_nan.replace(-1, np.nan, inplace=True)
corr_matrix = lfs_data_with_nan.corr()

strong_correlations = []
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)): 
        corr_value = corr_matrix.iloc[i, j]
        if (0.5 < corr_value < 1) or (-1 < corr_value < -0.5):
            strong_correlations.append((
                corr_matrix.index[i], 
                corr_matrix.columns[j], 
                corr_value
            ))

strong_correlations.sort(key=lambda x: abs(x[2]), reverse=True)

print("Strong correlations (|corr| > 0.5 and |corr| < 1):")
for var1, var2, corr in strong_correlations:
    print(f"{var1} — {var2}: {corr:.3f}")


We will use feature_cols as our predictor variables for PUFC11_WORK. Using Regression, we will evaluate how well we can predict whether or not someone has worked in the past week. We begin with preprocessing.

### Logistic Regression (LR)
Because this is a classification task, we can choose Logistic Regression as a baseline and a first model. In theory, PUFC11_WORK is a binary classification task, something that Logistic Regression should excel at. Since this uses a sigmoid function to map linear probabilities, between 0 and 1, making this suitable for determining whether someone "worked" (1) or "did not work" (0) within the past week. Finally, LR is particularly interpetable considering the task we're doing, allowing us to check the significance and coefficients associated with each predictor variable if need be.

#### Preprocessing: LR
We create a copy of our target variable PUFC11_WORK, and create five 80-20 train-test splits for the non-empty PUFC11 data we have. We'll also perform one-hot encoding, given our feature columns are all categorical. We also map the values of PUFC11, normally [1,2], to [0,1]. In addition, because we've determined that our feature_cols are intentionally unanswered as appropriate, we will be treating these values as empty rather than imputing.

In [ ]:
from src.preprocessing import prepare_data_kfold

target_col = 'PUFC11_WORK'
feature_cols = [
    'PUFC05_AGE', 'PUFC06_MSTAT', 'PUFC04_SEX', 
    'PUFC07_GRADE', 'PUFC08_CURSCH', 
    'PUFC38_PREVJOB', 'PUFC31_FLWRK',
    'PUFC30_LOOKW', 'PUFC34_WYNOT'
]

categorical_cols = feature_cols
n_splits = 5
missing_value =-1
seed = 45

folds_data = prepare_data_kfold(lfs_data, target_col = target_col,
                         n_splits = n_splits,
                         missing_value = missing_value,
                         categorical_cols = categorical_cols,
                         feature_cols = feature_cols,
                         seed = seed)

#### Training: LR
We are ready to start training. As we are setting a baseline, we'll start off with these hyperparameters. We'll use Stochastic Gradient Descent as our specified optimizer, though technically implementing mini-batch with a batch size of 128. We are looking for improvements in loss greater than 0.0001, else we stop early within three epochs. Lastly, we use a weight_decay of 0, indicating no regularization for now. All hyperparameters indicated below also indicate their default value if unspecified. For each convergence, we will track the metrics of that fold and aggregate it across all folds.

In [ ]:
from src.trainEval import *
result_dict = train_model(
    folds_data,
    optimizer_string="sgd", #Stochastic Gradient Descent
    scheduler_step_size=5,
    learning_rate=0.01,
    scheduler_gamma=0.5,
    convergence_threshold=1e-4, 
    num_epochs=50,
    patience=3, # Stop at 3 epochs with no improvement
    weight_decay=0,
    seed=seed
)


print("\nModel training complete!")
aggregate_cm = result_dict["aggregate_confusion_matrix"]

sns.heatmap(aggregate_cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=["Negative", "Positive"], 
            yticklabels=["Negative", "Positive"]) 


print(f"  Average Final Train Loss:     {result_dict['aggregated_final_metrics']['avg_final_train_loss']:.6f}")
print(f"  Average Final Test Loss:      {result_dict['aggregated_final_metrics']['avg_final_test_loss']:.6f}")
print(f"  Average Final Train Accuracy: {result_dict['aggregated_final_metrics']['avg_final_train_accuracy']:.6f}")
print(f"  Average Final Test Accuracy:  {result_dict['aggregated_final_metrics']['avg_final_test_accuracy']:.6f}")

plt.title("Aggregate Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

#### Error: LR
This model performed well for an initial run. All folds ran for 25 epochs. Both accuracies for test and train data are high, though there is usually minimal improvement from the third epoch onwards. The train and test loss both see steady improvement, but we see the test loss  to be slightly higher and plateau earlier at around the sixteenth epoch. 

We see that the train loss is slightly lower than the test loss in folds 1, 2, and 5, while the reverse is true for folds 3 and 4. This would indicate a model that overfits for the former folds, and underfits for the latter. Averaging these out, its clear that with an aggregated train and test loss of 0.684 and 0.687 respectively, the overall performance of the model doesn't significantly overfit nor underfit. This outcome is surprising, considering that we've used nine predictor variables while also forgoing the use of regularization, as these would have been expected to introduce noise and contribute to overfitting through curse of dimensionality.

#### Hyperparameter Tuning: LR

With our base parameters having been set, we can begin hyperparameter tuning. We'll implement *Random Search* as to not have to exhaustively branch through each possible combination of parameters. Below are the hyperparameter's we'll use and choose from. For example, if our algorithm randomly, for instance, chooses the second element of each, then it will test Logistic Regression at a learning rate of 0.1, batch size of 64, and the with the ADAM optimizer, etc. 

Values evenly spaced between 0.000001 and 0.001 will be explored for the learning rates. This will go on for fifty iterations, afterwhich a best model will be selected based on test accuracy. We'll perform random search rather than grid search, as a compromise to searching through all 259,200 combinations of hyperparameters in our grid.

In [ ]:
param_distributions = {
    'learning_rate': np.logspace(-4, -1, 20),
    'batch_size': [32, 64, 128, 256],
    'optimizer': ['sgd', 'adam', 'rmsprop'],
    'weight_decay': np.logspace(-5, -2, 10),
    'num_epochs': [30, 50, 75, 100],
    'scheduler_step_size': [5, 10, 15],
    'scheduler_gamma': [0.5, 0.7, 0.9],
    'patience': [3, 5, 7]
}

hyperparameter_results = hyperparameter_random_search(folds_data=folds_data, param_distributions=param_distributions, n_iter_search=50, seed=seed) 

# HPT Interpretation

In [ ]:
##best_params = hyperparameter_results['best_params']
##results_summary = hyperparameter_results['summary_df']


## Decision Tree (DT)

Another machine learning model we can use is the decision tree. Decision trees don't necessarily assume a linear relationship between our predictor variables and PUFC11, which could be benificial for finding complex relationships. Rather, they recursively partition our data with rules. Further, while it can get quite complex as it scales, individual nodes are also uncomplicated in how they are interpreted, which is also helpful for understanding the factors that determine whether an individual has worked within the past week.

#### Training: DT
We can recycle the one-hot encoded data we used in preparation for Logistic Regression and immediately use it for training our decision tree.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, log_loss, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
import numpy as np

accuracies_train = []
losses_train = []
accuracies_test = []
losses_test = []
confusion_matrices = [] 

for i in range(5):
    X_train, X_test, y_train, y_test = (
        folds_data[i]['X_train'],
        folds_data[i]['X_test'],
        folds_data[i]['y_train'],
        folds_data[i]['y_test'],
    )

    model = DecisionTreeClassifier(random_state=45, min_impurity_decrease=0.001)

    model.fit(X_train, y_train)

    # Train predictions and loss
    y_train_pred = model.predict(X_train)
    y_train_pred_proba = model.predict_proba(X_train)
    accuracy_train = accuracy_score(y_train, y_train_pred)
    loss_train = log_loss(y_train, y_train_pred_proba)

    # Test predictions and loss
    y_test_pred = model.predict(X_test)
    y_test_pred_proba = model.predict_proba(X_test)
    accuracy_test = accuracy_score(y_test, y_test_pred)
    loss_test = log_loss(y_test, y_test_pred_proba)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_test_pred)
    confusion_matrices.append(cm)

    accuracies_train.append(accuracy_train)
    losses_train.append(loss_train)
    accuracies_test.append(accuracy_test)
    losses_test.append(loss_test)

    print(f"Fold {i+1} - Train Accuracy: {accuracy_train:.4f}, Train Loss: {loss_train:.4f}, Test Accuracy: {accuracy_test:.4f}, Test Loss: {loss_test:.4f}")
    print(f"Fold {i+1} - Confusion Matrix:\n{cm}\n")

avg_accuracy_train = np.mean(accuracies_train)
avg_loss_train = np.mean(losses_train)
avg_accuracy_test = np.mean(accuracies_test)
avg_loss_test = np.mean(losses_test)

print(f"\nAggregated Metrics:")
print(f"Average Train Accuracy: {avg_accuracy_train:.4f}")
print(f"Average Train Loss: {avg_loss_train:.4f}")
print(f"Average Test Accuracy: {avg_accuracy_test:.4f}")
print(f"Average Test Loss: {avg_loss_test:.4f}")

print("\nOverall Confusion Matrix:")
print(sum(confusion_matrices))

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

#### Error: DT
Our model is also performing well, and with each confusion metric no less than 0.98. As this is a decision tree, a model usually prone to overfitting, we do see minimal overfitting, seeing as our train loss is higher than our test loss by 0.0023, though this is arguably a negligible gap.



#### Hyperparameter Tuning: DT

Similar to our methodology with Logistic Regression, we will implement random search for our tuning as a compromise to exhaustively searching through each hyperparameter combination. This time, we will explicitly define certain thresholds and measures. 

Our previous two hyperparameters, max_depth and minimum_impurity_decrease, will now range fom 30-60 and 0.001 to 0.1 respectively, adding variability to our parameters despite minimal signs of overfitting. We will allow the min_samples_leaf to enforce generalization or allow leaf nodes with one sample. 

Of note, we will add cost-complexity pruning as a form of regularizaton for our decision trees. We will also be exploring both gini and entropy as criteria, as the former minimizes chances of misclassification while the latter measures information gain.

#### Error: DT
Our model is also performing well, and with each confusion metric no less than 0.98. As this is a decision tree, a model usually prone to overfitting, we do see minimal overfitting, seeing as our train loss is higher than our test loss by 0.0023, though this is arguably a negligible gap.



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, log_loss

p_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [30, 45, 50, 60],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "min_impurity_decrease": [0.001, 0.005, 0.01],
    "ccp_alpha": np.logspace(-4, 0, 10)
}

best_params_list = []
accuracies = []
precisions = []
recalls = []
f1_scores = []
losses = []

for i in range(5):
    X_train, X_test, y_train, y_test = (
        folds_data[i]['X_train'],
        folds_data[i]['X_test'],
        folds_data[i]['y_train'],
        folds_data[i]['y_test'],
    )

    model = DecisionTreeClassifier(random_state=45)

    random_search = RandomizedSearchCV(
        model, p_grid, n_iter=20, cv=5, scoring="accuracy", n_jobs=-1, verbose=1, random_state=45
    )
    random_search.fit(X_train, y_train)

    best_params_list.append(random_search.best_params_)

    accuracies.append(accuracy_score(y_test, random_search.best_estimator_.predict(X_test)))
    report = classification_report(y_test, random_search.best_estimator_.predict(X_test), output_dict=True, zero_division=0)
    losses.append(log_loss(y_test, random_search.best_estimator_.predict_proba(X_test)))
    precisions.append(report['macro avg']['precision'])
    recalls.append(report['macro avg']['recall'])
    f1_scores.append(report['macro avg']['f1-score'])

    print(f"Fold {i+1} - Best Parameters: {random_search.best_params_}")
    print(f"Fold {i+1} - Accuracy: {accuracies[-1]:.4f}, Precision: {precisions[-1]:.4f}, Recall: {recalls[-1]:.4f}, F1-score: {f1_scores[-1]:.4f}, Loss: {losses[-1]:.4f}")


avg_accuracy = np.mean(accuracies)
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)
avg_f1 = np.mean(f1_scores)
avg_loss = np.mean(losses)

print(f"\nAggregated Metrics:")
print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1-score: {avg_f1:.4f}")
print(f"Average Loss: {avg_loss:.4f}")
print("\nBest Parameters for each fold:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i+1}: {params}")


best_fold_index = np.argmax(accuracies)
best_fold_params = best_params_list[best_fold_index]
print("\nBest Performing Fold Parameters:", best_fold_params)


best_model = DecisionTreeClassifier(random_state=45, **best_fold_params)


all_X_train = np.concatenate([folds_data[i]['X_train'] for i in range(5)])
all_y_train = np.concatenate([folds_data[i]['y_train'] for i in range(5)])

best_model.fit(all_X_train, all_y_train)


In [ ]:
from sklearn.metrics import confusion_matrix, log_loss, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

train_preds = best_model.predict(X_train)
train_acc = accuracy_score(y_train, train_preds)

test_preds = best_model.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)

train_probs = best_model.predict_proba(X_train)
test_probs = best_model.predict_proba(X_test)
train_loss = log_loss(y_train, train_probs)
test_loss = log_loss(y_test, test_probs)

cm = confusion_matrix(y_test, test_preds)  
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, 
            xticklabels=["Predicted Negative", "Predicted Positive"], 
            yticklabels=["Actual Negative", "Actual Positive"])


plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")


plt.show()

print(cm)

print(f"Training Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

print(f"Training Log Loss: {train_loss:.4f}")
print(f"Test Log Loss: {test_loss:.4f}")


train_precision = precision_score(y_train, train_preds, zero_division=0)
test_precision = precision_score(y_test, test_preds, zero_division=0)

train_recall = recall_score(y_train, train_preds, zero_division=0)
test_recall = recall_score(y_test, test_preds, zero_division=0)

train_f1 = f1_score(y_train, train_preds, zero_division=0)
test_f1 = f1_score(y_test, test_preds, zero_division=0)

print(f"Training Precision: {train_precision:.4f}")
print(f"Test Precision: {test_precision:.4f}")

print(f"Training Recall: {train_recall:.4f}")
print(f"Test Recall: {test_recall:.4f}")

print(f"Training F1-score: {train_f1:.4f}")
print(f"Test F1-score: {test_f1:.4f}")

# HPT Interpretation